# Exercice 28 : roulements à bille

## Combinaisons

Voici les différentes combinaisons possibles vérifiant $E - I - 2R = 0$:  
![png](combinaisons.png)

## Modélisation

Soit $x_i$ le nombre de roulements à bille produit avec la combinaison $i$. Par exemple $x_1$ consiste à utiliser un BE de rayon 300, un BI de rayon 260 et un BR de rayon 20.  
Les contraintes viennent des stocks. Par exemple, $x_1 + x_2 \leq 40$ vient du fait qu'il ne peut pas y avoir plus de 40 pièces de BE de rayon 300.


## Résolution avec Python

In [33]:
import mip

m = mip.Model()
m.verbose = 1
x = {i : m.add_var(name=f"x_{i}") for i in range(1, 14)}

def add_constraints(x, m):
    contraintes = [
        [1, 2, 40],         # coeff de x1, x2 et stock pour BE = 300
        [3, 4, 5, 6, 60],   # BE = 290
        [7, 8, 8, 10, 77],  # BE = 280
        [11, 12, 13, 55],   # BE = 274
        [1, 7, 30],         # BI = 260
        [3, 8, 35],         # BI = 256
        [2, 4, 11, 51],     # BI = 254
        [5, 12, 35],        # BI = 250
        [2, 4, 11, 51],     # BI = 254
        [10, 13, 34],       # BI = 240
        [7, 11, 26],        # BR = 10
        [8, 12, 25],        # BR = 12
        [3, 13, 75],        # BR = 17
        [4, 9, 50],         # BR = 18
        [1, 5, 10, 35],     # BR = 20
        [2, 6, 25],         # BR = 23
    ]
    for coeffs in contraintes:
        m += mip.xsum(x[i] for i in coeffs[:-1]) <= coeffs[-1] 

In [34]:
add_constraints(x, m)
m.objective = mip.maximize(mip.xsum(list(x.values())))
m.optimize()

tot = 0
for v in m.vars:
    print(f"{v.name}: {v.x}")
    tot += v.x
print(f"Total: {tot}")

x_1: 12.5
x_2: 25.0
x_3: 35.0
x_4: 0.0
x_5: 10.0
x_6: 0.0
x_7: 17.5
x_8: 0.0
x_9: 50.0
x_10: 12.5
x_11: 8.5
x_12: 25.0
x_13: 21.5
Total: 217.5


La solution obtenue n'est pas entière alors qu'on ne peut manifestement pas créer un nombre non-entier de roulements à bille ! Il faut modifier notre PL pour avoir un PLNE (programme linéaire en nombres entiers, où les variables sont entières):

In [37]:
m = mip.Model()
m.verbose = 1
x = {i : m.add_var(var_type=mip.INTEGER, name=f"x_{i}") for i in range(1, 14)}
add_constraints(x, m)
m.objective = mip.maximize(mip.xsum(list(x.values())))
m.optimize()

tot = 0
for v in m.vars:
    print(f"{v.name}: {v.x}")
    tot += v.x
print(f"Total: {tot}")

x_1: 12.0
x_2: 25.0
x_3: 35.0
x_4: 0.0
x_5: 10.0
x_6: 0.0
x_7: 18.0
x_8: 0.0
x_9: 50.0
x_10: 13.0
x_11: 8.0
x_12: 25.0
x_13: 21.0
Total: 217.0


Conclusion : le nombre maximum de roulements à bille que l'on peut produire est **217**. 